In [1]:
import sys
sys.path.append('../30_data_tools/')

In [2]:
import pandas as pd
import sqlite3
from tqdm.auto import tqdm
import re
import json

In [3]:
from helper import load_dotenv

In [5]:
from get_labelstudio_data import get_results_of_project

In [4]:
dotenv = load_dotenv()
con = sqlite3.connect( dotenv['DB_PATH'] )

In [8]:
masks = pd.read_sql(
    'SELECT * FROM mask',
    con
)

adjustments_per_mask = pd.read_sql(
    'SELECT * FROM adjustment_per_mask',
    con
)

In [ ]:
pkl_files = list((dotenv['GENERIC_DATA_DIR'] / "02_information_data").glob('./*.pkl'))
pkl_data = pd.concat([pd.read_pickle(pkl) for pkl in pkl_files], ignore_index=True)
pkl_data.loc[:,'job'] = pkl_data.img_path.apply(lambda val: val.parent.parent.name)
pkl_data.loc[:,'variant_name'] = pkl_data.img_path.apply(lambda val: val.parent.name)
pkl_data.loc[:,'pdf_filename'] = pkl_data.apply(lambda row: row.basic_name.replace( f'{ row.job }.','').split('.$PLACEHOLDER$.')[0], axis=1 )
pkl_data.loc[:,'idx'] = pd.to_numeric(pkl_data.apply(lambda row: row.basic_name.split('.$PLACEHOLDER$.')[1], axis=1 ))
pkl_data.loc[:,'type'] = '4c'

In [ ]:
mask_ids_to_update = adjustments_per_mask.loc[
    adjustments_per_mask.mask_id.isin(masks.mask_id) == False,
    ['job','pdf_filename','type','variant_name','method','idx','mask_id']
].drop_duplicates()

In [ ]:
mask_ids_to_update.shape

In [ ]:
matchings = []

for i in tqdm(range(mask_ids_to_update.shape[0])):
    row = mask_ids_to_update.iloc[i]
    
    candidates = masks.loc[
        (masks.job == row.job) &
        (masks.pdf_filename == row.pdf_filename) &
        (masks.type == row.type) &
        (masks.variant_name == row.variant_name) &
        (masks.method == row.method) &
        (masks.idx == row.idx) &
        (masks.mask_id.isin(adjustments_per_mask.mask_id) == False)
    ]
    
    pkl_candidates = pkl_data.loc[
        (pkl_data.job == row.job) &
        (pkl_data.pdf_filename == row.pdf_filename) &
        (pkl_data.type == row.type) &
        (pkl_data.variant_name == row.variant_name) &
        (pkl_data.method == row.method) &
        (pkl_data.idx == row.idx)
    ]
    
    relevant_idx = adjustments_per_mask.loc[
        adjustments_per_mask.mask_id == row.mask_id
    ].index
    
    for ridx in relevant_idx:
        pkl_candidates = pkl_candidates.loc[
            pkl_candidates[f'use_{adjustments_per_mask.loc[ridx].adjustment}']
        ]
    
        features = json.loads(adjustments_per_mask.loc[ridx].features)
    
        for key in features:
            if key in ['blow_up_radius','blow_up_c','contract_radius','contract_c']:
                pkl_candidates = pkl_candidates.loc[
                    pkl_candidates[key].apply(lambda val: str(val)) == str(features[key])
                ]
            elif key in ['blow_up_center','contract_center']:
                pkl_candidates = pkl_candidates.loc[
                    pkl_candidates[key].apply(lambda field: ";".join([",".join([str(val) for val in entry]) for entry in field])) == ";".join([",".join([str(val) for val in entry]) for entry in features[key]])
                ]
            else:
                pkl_candidates = pkl_candidates.loc[
                    pkl_candidates[key] == features[key]
                ]
    
    if pkl_candidates.shape[0] == 1:
        pkl_candidate = pkl_candidates.iloc[0]
    
        candidates = candidates.loc[
            candidates.bbox == ";".join([str(val) for val in pkl_candidate.bbox])
        ]
    
        if candidates.shape[0] > 0:
            matchings.append((
                row.mask_id,
                candidates.iloc[0].mask_id
            ))

In [ ]:
for source,target in matchings:
    c = con.cursor()
    try:
        c.execute(
            f'''
                UPDATE adjustment_per_mask
                SET mask_id='{ target }'
                WHERE mask_id='{ source }'
            '''
        )
    except:
        pass
    c.close()
    con.commit()

In [36]:
def _masks():
    with sqlite3.connect( dotenv['DB_PATH'] ) as con:
        # Masken laden
        masks = pd.read_sql(
            '''
                SELECT * FROM mask m
            ''',
            con
        )
        
        relevant_results = [
            r for r in
            get_results_of_project(2)
            if 'id' in r and r['id'] not in masks.mask_id.unique()
        ]
    
        # update masken filtern
        requires_update = []
    
        for r in tqdm(relevant_results):
            job, pdf_filename, variant_name, method, idx = re.match('(.+?)\.(.+\.p\d+)\.(halftone\d+dpi)\.(.+?).(\d+)\..+', r['img_name']).groups()
            idx = int(idx)
            bbox = f"{r['bbox']['x']};{r['bbox']['y']};{r['bbox']['width']};{r['bbox']['height']}"
            
            masks_selection = masks.loc[
                (masks.job == job) &
                (masks.pdf_filename == pdf_filename) &
                (masks.variant_name == variant_name) &
                (masks.method == method) &
                (masks.idx == idx) &
                (masks.bbox == bbox)
            ]
            
            if masks_selection.shape[0] > 0:
                requires_update.append((masks_selection.iloc[0].name,r['id']))
                    
        for update_line in tqdm(requires_update):
            row = masks.loc[update_line[0]]

            sql_mask = f'''
                UPDATE mask
                SET mask_id='{ update_line[1] }'
                WHERE (
                    pdf_filename='{ row.pdf_filename }' AND 
                    job='{ row.job }' AND 
                    "type"='{ row['type'] }' AND
                    variant_name='{ row.variant_name }' AND
                    idx={ row.idx } AND
                    mask_id='{ row.mask_id }'
                )
            '''

            sql_adjustments = f'''
                UPDATE adjustment_per_mask
                SET mask_id='{ update_line[1] }'
                WHERE (
                    pdf_filename='{ row.pdf_filename }' AND 
                    job='{ row.job }' AND 
                    "type"='{ row['type'] }' AND
                    variant_name='{ row.variant_name }' AND
                    idx={ row.idx } AND
                    mask_id='{ row.mask_id }'
                )
            '''
            c = con.cursor()
            try:
                c.execute( sql_mask )
                c.execute( sql_adjustments )
            except sqlite3.IntegrityError:
                pass
                
            c.close()
            con.commit()

In [42]:
_masks()

  0%|          | 0/73 [00:00<?, ?it/s]

  0%|          | 0/69 [00:00<?, ?it/s]